# Sentence Transformer Evaluation

In [ ]:
from sentence_transformers import SentenceTransformer
import os
from triplet_semantic_similarity_sbert import form_triplets, load_doc_col, split_doc_col, form_triplets
from enum import Enum

doc_col = load_doc_col()

queries = {(1,
            "Right To Information"): "Das Bestehen eines Rechts auf Auskunft seitens des Verantwortlichen über die "
                                     "betreffenden personenbezogenen Daten.",
           (2,
            "Right To Rectification Or Deletion"): "Die betroffene Person hat das Recht, von dem Verantwortlichen "
                                                   "unverzüglich die Berichtigung sie betreffender unrichtiger "
                                                   "personenbezogener Daten zu verlangen. Die betroffene Person hat "
                                                   "das Recht, von dem Verantwortlichen zu verlangen, dass sie "
                                                   "betreffende personenbezogene Daten unverzüglich gelöscht werden.",
           (3,
            "Right To Data Portability"): "Die betroffene Person hat das Recht, die sie betreffenden personenbezogenen "
                                          "Daten, die sie einem Verantwortlichen bereitgestellt hat, in einem "
                                          "strukturierten, gängigen und maschinenlesbaren Format zu erhalten.",
           (4, "Right To Withdraw Consent"): "Das Bestehen eines Rechts, die Einwilligung jederzeit zu widerrufen.",
           (5, "Right To Complain"): "Das Bestehen eines Beschwerderechts bei einer Aufsichtsbehörde."}

class ModelPath(Enum):
    
    1 = ""

In [ ]:
query_id = 1
query = queries[query_id]
test_docs, train_docs = split_doc_col(doc_col, query_id)
test_triplets = form_triplets(query_id, test_docs)
model = SentenceTransformer.load(os.path.join(os.path.dirname(__file__), ModelPath(query_id)))